## imports

In [1]:
# imports from core
import statistics
import os
import re
import math
import operator
import sys
from functools import partial
import inspect
import logging
import traceback as tb

# imports regarding stats
import pandas as pd
from pandas.tseries.offsets import *
from pandas.plotting import table
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.offsetbox import AnchoredText
from matplotlib.ticker import FuncFormatter
from matplotlib.gridspec import GridSpec
from datetime import datetime
from scipy import stats


import pdb

## set logger

In [2]:
# get logger
logger = logging.getLogger(__name__)

# set logging level
logger.setLevel(logging.DEBUG)

# set up log level in file
file_handler = logging.FileHandler('.log')

# set logging level on file handler
file_handler.setLevel(logging.DEBUG)

# create a logging format
formatter = logging.Formatter('%(asctime)s;%(levelname)s;%(message)s;')

# set the formatter
file_handler.setFormatter(formatter)

# add the file handler to logger
logger.addHandler(file_handler)

## initialize variables
# set initial comment
logger.info('Starting....')

## set parameters

In [3]:
# set style
plt.style.use('mccm-1')
# annotate plots
annot=False
# set audience type (internal=True; external=False)
audience_internal=True
# set whether files will be saved
save_files=True
# set default folder name
prefix='images'
# rename print function
_p=print
# function to show plots, and save if asked
def show(filename=None, folder='images', display=True, tight_layout=True):
    # make figure tighter
    fig.tight_layout() if tight_layout else None
    # show the plot
    plt.show() if display else None 
    # has a filename been passed
    if filename:
        # yes, save the image
        fig.savefig(folder+'/'+filename)
    # clear the figure to release memory
    plt.clf()

# Combine data from individual manager tabs  

In [4]:
# file name
file_name = 'ncf_input_data_01.xlsx'

# absolute file path
file_path = '/Users/darpanbiswas/Dropbox/01_MCCM/89_DATA/{}'.format(file_name)



In [5]:
investment_details_tab = [
    'Co-investments',
    'Clearvue II',
    'Quan', 
    'AlphaX',
    'Quan II',
    'TF III',
    'XiangHe II',
    
]

revenue_income_tab = [
    'Clearvue II PC',
    'Quan PC', 
    'AlphaX PC',
    'Quan II PC',
    'TF III PC',
    'XiangHe II PC',  
    'Huolala'
]

In [6]:
def get_empty_combined_manager_holdings_df():
    
    _temp_df = pd.DataFrame(
    [],
    columns=[
                            'Realization Status', 
                            'Portfolio Company',
                            'Date',
                            'Year',
                            'Sector',
                            'Strategy',
                            'Cost',
                            'Stake',
                            'Realized',
                            'Unrealized',
                            'Multiple',
                            'IRR',
                            'Exits',
                            'Stage',
                            'Geography',
                            'Main Sector',
                            'Fund',
                            'NCF Committed Capital',
                            'Fund Size',
                            'FMV',       
                            'NCF Cost',
                            'NCF Realized', 
                            'NCF Unrealized', 
                            'NCF Value', 
                        ])
    
    return _temp_df

In [7]:
def get_empty_combined_manager_investment_pace_df():
    
    _temp_df = pd.DataFrame(
    [],
    columns=[
        'Portfolio Company',
        'Date',
        'Year',
        'Cost',
        'Fund',]
    )
    
    return _temp_df

In [8]:
def get_empty_combined_manager_holdings_financials_df():
    
    _temp_df = pd.DataFrame(
        [],
        columns=['Company'] + ['{}'.format(year) for year in range(2015,2019)] + ['2019Q1'] + ['Fund']
    )
    
    return _temp_df

In [9]:
def get_manager_parameter(*args, manager_df, parameter_name, **kwargs):

    parameter_locators = {
        'sector':[1,0],
        'ncf_committed_capital':[0,4],
        'manager_fund_size':[0,5],
    }
    
    try:
        
        parameter_value = manager_df.iloc[
            parameter_locators[parameter_name][0], 
            parameter_locators[parameter_name][1],
        ]
        
        assert parameter_value, 'parameter_not_found'
                
        return parameter_value
    
    except AssertionError as ae:
     
        logger.info('...could not find:{0};{1}'.format(parameter_name, ae))
        
        return 'error'       
        
    except Exception as e:
    
        logger.info('...error finding parameter:{0};{1}'.format(parameter_name, e))
        
        return 'error'
    

In [10]:
def return_realized_unrealized_string_array(*args, manager_df, **kwargs):
    
    re_unreal = re.compile('Unrealized.*')
    
    re_total = re.compile('Total.*')
    
    string_array = manager_df.iloc[:,0].values.tolist()
    
    reached_total_row = False
    
    current_group = ''
    
    for index in range(len(string_array)):
        
        _temp = string_array[index]
        
        if reached_total_row:
            
            string_array[index] = 'drop'

        else:

            if type(string_array[index]) == str:
                
                if re_unreal.match(current_group) and re_total.match(string_array[index]):
                
                    reached_total_row = True
                    
                current_group = string_array[index]

                string_array[index] = 'drop'

            else:

                string_array[index] = current_group

#         print(_temp, string_array[index])

    return string_array        

In [11]:
# function to parse manager name
def parse_manager_tab_investments(*args, combined_manager_holdings_df, manager_name, **kwargs):
    
    # try to read the tab
    try:
        
        # log that the program is starting on a tab
        logger.info('...parsing manager tab for:{}'.format(manager_name))
    
        # read the excel file
        manager_df = pd.read_excel(file_path, sheet_name=manager_name, header=None, skiprows=[0]).iloc[:, 1:]

        # get sector, committed capital, and find size data
        sector, ncf_committed_capital, manager_fund_size = \
            [get_manager_parameter(
                manager_df = manager_df, 
                parameter_name = parameter_name,
            ) for parameter_name in ['sector', 'ncf_committed_capital', 'manager_fund_size']]

        # remove first two rows
        manager_df = manager_df.iloc[2:, :]

        # set columns
        manager_df.columns = manager_df.iloc[0]

        # rename the first column
        manager_df.columns = ['Realization Status'] + manager_df.columns[1:].values.tolist()

        # drop row that was set as column
        manager_df = manager_df.iloc[1:, :]

        # update first column
        manager_df.iloc[:, 0] = return_realized_unrealized_string_array(manager_df=manager_df)

        # drop rows using the realized / unrealized column
        manager_df = manager_df.loc[manager_df.iloc[:,0] != 'drop',:]

        # add column to indicate manager name
        manager_df.loc[:,'Fund'] = None; manager_df.loc[:,'Fund']=manager_name
        
        # add column to indicate year
        manager_df.loc[:, 'Year'] = manager_df.Date.min().year

        # add column to show NCF Committed Capital
        manager_df.loc[:, 'NCF Committed Capital'] = None; 
        manager_df.loc[:, 'NCF Committed Capital'] = ncf_committed_capital

        # add column to show Fund Size
        manager_df.loc[:, 'Fund Size'] = None; 
        manager_df.loc[:, 'Fund Size'] = manager_fund_size

        # add column to compute FMV at manager level
        manager_df.loc[:, 'FMV'] = None; 
        manager_df.loc[:, 'FMV'] = manager_df.loc[:, 'Realized'] + manager_df.loc[:,'Unrealized']

        # add column to compute FMV at manager level
        manager_df.loc[:, 'NCF Cost'] = None; 
        manager_df.loc[:, 'NCF Cost'] = manager_df.loc[:, 'NCF Committed Capital']/manager_df.loc[:, 'Fund Size']*manager_df.loc[:,'Cost']

        # add column to compute FMV at manager level
        manager_df.loc[:, 'NCF Realized'] = None; 
        manager_df.loc[:, 'NCF Realized'] = manager_df.loc[:, 'NCF Committed Capital']/manager_df.loc[:, 'Fund Size']*manager_df.loc[:,'Realized'] 

        # add column to compute FMV at manager level
        manager_df.loc[:, 'NCF Unrealized'] = None; 
        manager_df.loc[:, 'NCF Unrealized'] = manager_df.loc[:, 'NCF Committed Capital']/manager_df.loc[:, 'Fund Size']*manager_df.loc[:,'Unrealized'] 

        # add column to compute FMV at manager level
        manager_df.loc[:, 'NCF Value'] = None; 
        manager_df.loc[:, 'NCF Value'] = manager_df.loc[:, 'NCF Committed Capital']/manager_df.loc[:, 'Fund Size']*manager_df.loc[:,'FMV']    
        
        # combine the original df to the new df
        _temp_combined_df = combined_manager_holdings_df.append(manager_df)
                
        # log that the program is done on a tab
        logger.info('...done parsing manager tab for:{}'.format(manager_name))

        # return the update df
        return _temp_combined_df
    
    # catch exception as e
    except Exception as e:
    
        # log the error
        logger.info('...unable to finish finding investments data; error:{}'.format(tb.format_exc()))
        
        # return the error df
        return combined_manager_holdings_df

In [12]:
# function to parse manager name
def parse_manager_tab_investment_pace(*args, combined_manager_investment_pace_df, manager_name, **kwargs):
    
    # try to get the investment pace
    try:

        # read the excel file
        manager_df = pd.read_excel(file_path, sheet_name=manager_name, header=None).iloc[:, 12:]

        # get the row that contains the word "Portfolio Company"
        _idx = manager_df.loc[manager_df.iloc[:,0]=='Portfolio Company'].index

        # 
        manager_df = manager_df.iloc[_idx.values[0]:].dropna()

        # set column names
        manager_df.columns = manager_df.iloc[0]

        # drop the first row
        manager_df = manager_df.iloc[1:]

        # set Fund name
        manager_df.loc[:,'Fund'] = manager_name
        
        # combine the original df to the new df
        _temp_combined_df = combined_manager_investment_pace_df.append(manager_df)
                        
        # log that the program is done on a tab
        logger.info('...done parsing manager investment pace for:{}'.format(manager_name))

        # return the update df
        return _temp_combined_df
        
    # catch exception as e
    except Exception as e:
    
        # log the error
        logger.info('...unable to finish finding investment pace data; error:{}'.format(tb.format_exc()))
        
        # return the error df
        return combined_manager_investment_pace_df

In [13]:
# some of the column headers are integers or floats for some reason; change to str
def change_column_header_to_string(*args, column_header, **kwargs):
                
    # try to return int representation
    try:
        
        # convert to int
        return repr(int(column_header))
    
    # could not convert to int
    except Exception as e:
                
        # return as is
        return column_header    

# parse manager revenue tab
def parse_manager_pc_tab(*args, combined_manager_holdings_revenue_df, combined_manager_holdings_income_df, manager_name, **kwargs):
    
    # try to parse the pc tab
    try:

        # get the data in the tap
        manager_df = pd.read_excel(file_path, sheet_name='{}'.format(manager_name), header=None)

        # get the 6th row
        _temp_row = manager_df.iloc[5]

        # find columns with "No." in the 6th row
        anchor_columns = _temp_row[_temp_row == 'No.'].index.values.tolist()

        # calcuate the number of periods
        num_of_periods = anchor_columns[1] - anchor_columns[0] - 3
        
        # get columns 3 and 4
        _temp_columns = manager_df.iloc[5:, [2, 3]]

        # find rows where the first column is not an int
        _temp_matches = _temp_columns.apply(lambda row:type(row.iloc[0]) == int, axis=1)

        # get index of last match
        number_of_companies = _temp_columns[_temp_matches].index.shape[0]
        
        # get revenue data
        revenue_data = manager_df.iloc[5:5+(number_of_companies+1), 3:(4+num_of_periods)]
                
        # update header
        revenue_data.columns = revenue_data.iloc[0].apply(lambda column:change_column_header_to_string(column_header=column))

        # drop first row
        revenue_data = revenue_data.iloc[1:]

        # get income data
        income_data = manager_df.iloc[5:5+(number_of_companies+1), anchor_columns[1]+1:(anchor_columns[1]+2+num_of_periods)]

        # update header
        income_data.columns = income_data.iloc[0].apply(lambda column:change_column_header_to_string(column_header=column))

        # drop first row
        income_data = income_data.iloc[1:]
        
        # add fund column
        revenue_data.loc[:, 'Fund'] = manager_name; income_data.loc[:, 'Fund'] = manager_name
                
        # combine revenue data
        _temp_revenue_df = combined_manager_holdings_revenue_df.append(revenue_data)
        
        # combined income data
        _temp_income_df = combined_manager_holdings_income_df.append(income_data) 
                                
        # log that the program is done on a tab
        logger.info('...done parsing manager pc tab for:{}'.format(manager_name))

        # return both revenue and income data
        return [_temp_revenue_df, _temp_income_df]
    
    # something went wrong
    except Exception as e:        
            
        # log the error
        logger.info('...unable to finish finding revenue and income data; error:{}'.format(tb.format_exc()))
        
        # return original df
        return [combined_manager_holdings_revenue_df, combined_manager_holdings_income_df]

In [ ]:
combined_manager_holdings_df = get_empty_combined_manager_holdings_df()

combined_manager_investment_pace_df = get_empty_combined_manager_investment_pace_df()

combined_manager_holdings_revenue_df = get_empty_combined_manager_holdings_financials_df()

combined_manager_holdings_income_df = get_empty_combined_manager_holdings_financials_df()

for manager_name in investment_details_tab:
    
    sys.stdout.write('processing manager:{:20}\n'.format(manager_name)); logger.info('processing manager:{}'.format(manager_name))

    combined_manager_holdings_df = parse_manager_tab_investments(
        combined_manager_holdings_df=combined_manager_holdings_df, 
        manager_name=manager_name,
    )
        
    combined_manager_investment_pace_df = parse_manager_tab_investment_pace(
        combined_manager_investment_pace_df = combined_manager_investment_pace_df,
        manager_name=manager_name
    )
    

for manager_name in revenue_income_tab:
    
    combined_manager_holdings_revenue_df, combined_manager_holdings_income_df = parse_manager_pc_tab(
        combined_manager_holdings_revenue_df=combined_manager_holdings_revenue_df,
        combined_manager_holdings_income_df=combined_manager_holdings_income_df,
        manager_name=manager_name,
    )

In [ ]:
combined_manager_holdings_df = combined_manager_holdings_df.reset_index(drop=True)

combined_manager_holdings_df.index.name = 'row_id'

combined_manager_holdings_df = combined_manager_holdings_df.rename(
    {
        'NCF Value':'NCF FMV',
    }, 
    axis=1)

combined_manager_holdings_df = combined_manager_holdings_df.loc[:,[
            'Fund',
            'Geography',
            'Portfolio Company',
            'Date',
            'Year',
            'Sector',
            'Main Sector',
            'Strategy',
            'Cost',
            'Fund Size',
            'NCF Committed Capital',
            'NCF Cost',
            'Stake',
            'Realized',
            'NCF Realized',
            'Unrealized',
            'NCF Unrealized',
            'FMV',
            'NCF FMV',
            'Multiple',
            'IRR',
            'Exits',
            'Stage',
            'Realization Status',
]]

In [ ]:
# reorder columns to put Fund and Portfolio Company up front
def put_fund_and_portfolio_company_columns_up_front(*args, columns, reorder_column_names, **kwargs):

    # get all the columns
    _temp_columns =  columns.values.tolist()

    # remove Fund and Portfolio Company
    for column in reorder_column_names: _temp_columns.remove(column)
                                                             
    # add them back to the front
    _temp_columns = reorder_column_names + _temp_columns
    
    # return data
    return _temp_columns

In [ ]:
combined_manager_investment_pace_df = combined_manager_investment_pace_df.reset_index(drop=True)

combined_manager_investment_pace_df.loc[:, 'Year'] = combined_manager_investment_pace_df.Date.apply(lambda row:row.year)

combined_manager_investment_pace_df.index.name = 'row_id'

# get re-ordered coumns
combined_manager_investment_pace_df = combined_manager_investment_pace_df.loc[
    :,
    put_fund_and_portfolio_company_columns_up_front(
        columns=combined_manager_investment_pace_df.columns,
        reorder_column_names = ['Fund', 'Portfolio Company'])]

In [ ]:
# reset the index
combined_manager_holdings_revenue_df = combined_manager_holdings_revenue_df.reset_index(drop=True)

# change the index name
combined_manager_holdings_revenue_df.index.name = 'row_id'

# get re-ordered coumns
combined_manager_holdings_revenue_df = combined_manager_holdings_revenue_df.loc[
    :,
    put_fund_and_portfolio_company_columns_up_front(
        columns=combined_manager_holdings_revenue_df.columns,
        reorder_column_names = ['Fund', 'Company'])]

In [ ]:
# reset the index
combined_manager_holdings_income_df = combined_manager_holdings_income_df.reset_index(drop=True)

# change the index name
combined_manager_holdings_income_df.index.name = 'row_id'

# get re-ordered coumns
combined_manager_holdings_income_df = combined_manager_holdings_income_df.loc[
    :,
    put_fund_and_portfolio_company_columns_up_front(
        columns=combined_manager_holdings_revenue_df.columns,
        reorder_column_names = ['Fund', 'Company'])]

In [ ]:
# create an excel writer
excel_writer = pd.ExcelWriter('combined_{}'.format(file_name))

# save the combined holdings
combined_manager_holdings_df.to_excel(excel_writer,'combined_holdings')

# save the combined investment pace
combined_manager_investment_pace_df.to_excel(excel_writer, 'combined_investment_pace')

# save the combined revenue
combined_manager_holdings_revenue_df.to_excel(excel_writer, 'combined_revenue')

# save the combined income
combined_manager_holdings_income_df.to_excel(excel_writer, 'combined_income')

# write to disk
excel_writer.save()

# NCF portfolio diversification charts

In [ ]:
# absolute file path
file_path = '/Users/darpanbiswas/Dropbox/01_MCCM/89_DATA/ncf_input_data_01.xlsx'

# load xlsx sheet
ncf_holdings_level_2 = pd.read_excel(file_path, 'Sum')

# drop any columns with errors in all cells
ncf_holdings_level_2=ncf_holdings_level_2.dropna(how='all', axis=1)

In [ ]:
def redistribute_sectors(grouped_srs):

    redistributed_sums=[]

    for index, row in grouped_srs.iteritems():

        if '/' in index:

            split_indices = index.split('/')

            num_of_split_indices = len(split_indices)

            for _t_index in split_indices:

                redistributed_sums += [[_t_index, row/num_of_split_indices]]

        else:

            redistributed_sums += [[index, row]]
            
#     pdb.set_trace()
            
    redistributed_df = pd.DataFrame(redistributed_sums)
        
    redistributed_df.columns = ['Sector0PPT', 'NCF cost']
        
    _t_group = redistributed_df.groupby('Sector0PPT').sum().loc[:, 'NCF cost']
    
    return _t_group
        

In [ ]:
def pie_label(pct, *args, **kwargs):
    
    return "{:.1f}%".format(pct)

In [ ]:
pie_chart_params_set = [
    {'title':'By Asset Class', 'group_by_field':'Strategy'},
    {'title':'By Industry', 'group_by_field':'Sector0PPT'},
    {'title':'By Liquidity', 'group_by_field':'Stage'},
    {'title':'By Geography', 'group_by_field':'Geography'},
]
    
fig, axes = plt.subplots(4, 2, figsize=(20,15), gridspec_kw={'width_ratios':[0.5, 0.5], 'height_ratios':[0.05, 0.45, 0.05, 0.45]})

axes = [
    (axes[0,0], axes[1,0]), 
    (axes[0,1], axes[1,1]), 
    (axes[2,0], axes[3,0]), 
    (axes[2,1], axes[3,1]),
]

for pie_chart_param, (title_axis, chart_axis) in zip(pie_chart_params_set, axes):
        
    _t_grouped_data = ncf_holdings_level_2.groupby(pie_chart_param['group_by_field']).sum().loc[:, 'NCF cost']
    
    if (pie_chart_param['group_by_field'] == 'Sector0PPT'):
    
        _t_grouped_data = redistribute_sectors(_t_grouped_data)
        
    _t_grouped_data = _t_grouped_data.sort_values(0, ascending=False)
    
    _t_grouped_data = _t_grouped_data.loc[_t_grouped_data/_t_grouped_data.sum() > 0.01]
        
    labels = ['{}'.format(index) for index, row in _t_grouped_data.iteritems()]

    patches, texts, autotexts = chart_axis.pie(
        _t_grouped_data, 
        labels=labels,autopct=pie_label, 
        pctdistance=0.75,
        startangle=90,
        counterclock=False)
    
    for text, autotext in zip(texts, autotexts):
        
        text.set_fontsize(15)
        
        autotext.set_fontsize(12)       
    
    chart_axis.axis('equal')
    
    title_axis.get_xaxis().set_visible(False)

    title_axis.get_yaxis().set_visible(False)

    title_axis.set_facecolor('C1')
    
    at = AnchoredText(
        pie_chart_param['title'], 
        loc=10, 
        prop=dict(
            backgroundcolor='C1',
            size=20, 
            color='white',
            weight='bold'
        ))

    title_axis.add_artist(at)
                                                                                      
#     title = axis.set_title(pie_chart_param['title'], fontdict={'fontsize':20, 'fontweight':'bold', 'color':'white'})

#     title.set_bbox(dict(facecolor='C1', alpha=1.0, edgecolor='black', boxstyle='square', pad=0.4))
    
#     bbox_patch = title.get_bbox_patch()
    
#     bbox_patch.set_boxstyle("square", pad=0.0, width=axis.get_window_extent().width )
#     
#     bbox_patch.set_width(10)
    
#     bbox_patch.update()

    
fig.tight_layout()
    
plt.show()